In [1]:
import pandas as pd
import numpy as np
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import gc
import h5py

from PIL import Image
from pyts.image import RecurrencePlot, GramianAngularField
from pyts.datasets import load_gunpoint
from pyts.image import MarkovTransitionField
from pyts.image import GramianAngularField

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import tensorflow.python.keras.backend as K

from keras import optimizers
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Conv3D,Conv2D, MaxPooling3D, MaxPooling2D, Dropout, LSTM
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from keras.applications import VGG16
from keras.optimizers import Adam

Using TensorFlow backend.


# Make RP

In [ ]:
check = 1

path = 'E:/Data/ERP_array/RP/'
data = pd.read_csv('./input/ERPdata.csv', header=0)

for i in np.unique(data.subject):
    start = time.time()
    
    for j in np.unique(data[data.subject==i].condition):
        condition_data = data[(data.subject==i) & (data.condition==j)]
        condition_data = condition_data.iloc[:,2:11].T
        
        rp = RecurrencePlot(threshold=None, percentage=20)
        data_rp = rp.fit_transform(condition_data)
        
        np.savez_compressed(path+'/sub%d_con%d' %(i,j), data_rp)
    
    print("%d time : "%check, time.time() - start)
    check  = check+1

In [2]:
def make_input_size(data):
    img = Image.fromarray(data[0].astype('uint8'), 'L')
    img = img.resize((224,224))
    img_arr = np.asarray(img)
    result_arr = img_arr
    
    for i in range(1,9):
        img = Image.fromarray(data[i].astype('uint8'), 'L')
        img = img.resize((224,224))
        img_arr = np.asarray(img)
        result_arr = np.concatenate([result_arr, img_arr], axis=0)
    
    result_arr = result_arr.reshape(-1,224,224)
    return result_arr

In [ ]:
# make vgg data 1~9
path = 'E:/Data/ERP_array/RP/'
save_path = 'E:/Data/ERP_array/vgg_data/'
files = os.listdir(path)

for i in range(1,10):
    file_list = []
    
    for file in files:
        if (file[4] == '_') and 'sub'+str(i) in file:
            file_list.append(file)
    
    for file in file_list:
        if not os.path.isdir(save_path+'/%d'%i):
            os.mkdir(save_path+'/%d'%i)
        
        
        data = np.load(path+file)
        data = data['arr_0']
        input_data=(make_input_size(data))

        np.savez_compressed(save_path+str(i)+'/'+file, input_data)

In [ ]:
# make vgg data 10~81
path = 'E:/Data/ERP_array/RP/'
save_path = 'E:/Data/ERP_array/vgg_data/'
files = os.listdir(path)

for i in range(10,82):
    file_list = []
    
    for file in files:
        if 'sub'+str(i) in file:
            file_list.append(file)
    
    for file in file_list:
        if not os.path.isdir(save_path+'/%d'%i):
            os.mkdir(save_path+'/%d'%i)
        
        
        data = np.load(path+file)
        data = data['arr_0']
        input_data=(make_input_size(data))
        
        np.savez_compressed(save_path+str(i)+'/'+file, input_data)

In [3]:
#Learning Curve

def draw_plot(hist):
    fig, loss_ax = plt.subplots()

    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    acc_ax.set_ylabel('accuray')

    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')

    plt.show()

# Model

In [4]:
def create_model():
    pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    pre_trained_vgg.trainable = False

    model = Sequential()
    model.add(Conv2D(3, kernel_size=(3, 3),
                     activation='relu',
                     padding='same',
                     kernel_initializer='he_normal',
                     #kernel_regularizer=regularizers.l2(0.01),
                     #data_format = 'channels_first',
                     input_shape=(224,224,1)))
    model.add(pre_trained_vgg)
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    #model.add(Dropout(rate=0.2))
    model.add(Dense(2048, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(Dense(256, activation='relu'))
    #model.add(Dense(64, activation='relu'))
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
    
    #return model
    return model, model.get_weights()


# Make Input Data & Label

In [5]:
label_path = './input/demographic.csv'
label_data = pd.read_csv(label_path)
label_list = label_data['group']

In [6]:
path = 'E:/Data/ERP_array/vgg_data/'
x_data = []
y_data = []

for i in range(1,82):
    file_list = os.listdir(path+'%d' %i)
    #del_con(file_list)
    label = label_list[i-1]
    
    label = np.repeat(label, len(file_list))
    for k in range(9):
        y_data = np.append(y_data, label)
    
    for file in file_list:      
        data = np.load(path+str(i)+'/'+file)
        data = data['arr_0']
        x_data.append(data)
        
x_data = np.array(x_data)
y_data = np.array(y_data)
x_data = x_data.reshape(-1,224,224, 1)

In [7]:
x_data.shape

(2187, 224, 224, 1)

In [8]:
kfold = KFold(n_splits=10, shuffle=True, random_state=1004)

In [9]:
path = 'E:/Data/ERP_array/vgg_data/'

In [10]:
def init_weight(same_old_model, first_weights):
    same_old_model.set_weights(weights)

In [11]:
def cf_mat(test):
    y_pred = model.predict_classes(x_data[test])
    return confusion_matrix(y_data[test], y_pred)

In [12]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50, restore_best_weights=True)

# Train & Test

In [13]:
num = 0
k_accuracy = list()
accuracy = list()
mat = list()

model, weights = create_model()
for train, test in kfold.split(x_data):
    num = num+1
    print("---------------------------epoch : ",num,"---------------------------")
    init_weight(model, weights)
    model.fit(x_data[train], y_data[train], epochs=200, batch_size=20, callbacks=[es])
    accuracy =  (model.evaluate(x_data[test], y_data[test]))
    k_accuracy.append(accuracy)
    
    mat.append(cf_mat(test))

---------------------------epoch :  1 ---------------------------
Epoch 1/200
1968/1968 [==============================] - 20s 10ms/step - loss: 0.7721 - accuracy: 0.5727
Epoch 2/200
1968/1968 [==============================] - 16s 8ms/step - loss: 0.6755 - accuracy: 0.6138
Epoch 3/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.6197 - accuracy: 0.6662
Epoch 4/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.6060 - accuracy: 0.6839
Epoch 5/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.4931 - accuracy: 0.7693
Epoch 6/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.4539 - accuracy: 0.7861
Epoch 7/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.3747 - accuracy: 0.8308
Epoch 8/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.3370 - accuracy: 0.8526
Epoch 9/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.2526 - accuracy: 0.89

1968/1968 [==============================] - 15s 8ms/step - loss: 0.5585 - accuracy: 0.7119
Epoch 5/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.4614 - accuracy: 0.7779
Epoch 6/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.3973 - accuracy: 0.8227
Epoch 7/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.3306 - accuracy: 0.8547
Epoch 8/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.3240 - accuracy: 0.8476
Epoch 9/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.2469 - accuracy: 0.8852
Epoch 10/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.1916 - accuracy: 0.9182
Epoch 11/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.1764 - accuracy: 0.9248
Epoch 12/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.1475 - accuracy: 0.9405
Epoch 13/200
1968/1968 [==============================] - 15s 8ms/step - 

1968/1968 [==============================] - 15s 8ms/step - loss: 0.0119 - accuracy: 0.9949
Epoch 160/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0328 - accuracy: 0.9878
Epoch 161/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0305 - accuracy: 0.9893
Epoch 162/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0051 - accuracy: 0.9990
Epoch 163/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0032 - accuracy: 0.9990
Epoch 164/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0016 - accuracy: 0.9995
Epoch 165/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0312 - accuracy: 0.9878
Epoch 166/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0087 - accuracy: 0.9975
Epoch 167/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0058 - accuracy: 0.9980
Epoch 168/200
1968/1968 [==============================] - 1

1968/1968 [==============================] - 15s 8ms/step - loss: 0.0091 - accuracy: 0.9970
Epoch 113/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0085 - accuracy: 0.9975
Epoch 114/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0106 - accuracy: 0.9964
Epoch 115/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0372 - accuracy: 0.9883
Epoch 116/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0261 - accuracy: 0.9868
Epoch 117/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0112 - accuracy: 0.9949
Epoch 118/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0063 - accuracy: 0.9970
Epoch 119/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0146 - accuracy: 0.9959
Epoch 120/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0298 - accuracy: 0.9878
Epoch 121/200
1968/1968 [==============================] - 1

1968/1968 [==============================] - 15s 8ms/step - loss: 0.0090 - accuracy: 0.9985
Epoch 190/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0225 - accuracy: 0.9898
Epoch 191/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0365 - accuracy: 0.9853
Epoch 192/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0042 - accuracy: 0.9985
Epoch 193/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0034 - accuracy: 0.9990
Epoch 194/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0019 - accuracy: 0.9995
Epoch 195/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0112 - accuracy: 0.9959
Epoch 196/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0101 - accuracy: 0.9964
Epoch 197/200
1968/1968 [==============================] - 15s 8ms/step - loss: 0.0023 - accuracy: 0.9995
Epoch 198/200
1968/1968 [==============================] - 1

...

1968/1968 [==============================] - 21s 11ms/step - loss: 0.0049 - accuracy: 0.9990
Epoch 159/200
1968/1968 [==============================] - 21s 11ms/step - loss: 0.0183 - accuracy: 0.9924
Epoch 160/200
1968/1968 [==============================] - 21s 11ms/step - loss: 0.0133 - accuracy: 0.9949
Epoch 161/200
1968/1968 [==============================] - 21s 11ms/step - loss: 0.0274 - accuracy: 0.9919
Epoch 162/200
1968/1968 [==============================] - 21s 11ms/step - loss: 0.0179 - accuracy: 0.9939
Epoch 163/200
1968/1968 [==============================] - 22s 11ms/step - loss: 0.0088 - accuracy: 0.9964
Epoch 164/200
1968/1968 [==============================] - 21s 10ms/step - loss: 0.0190 - accuracy: 0.9939
Epoch 165/200
1968/1968 [==============================] - 21s 11ms/step - loss: 0.0259 - accuracy: 0.9929
Epoch 166/200
1968/1968 [==============================] - 21s 10ms/step - loss: 0.0049 - accuracy: 0.9980
Epoch 167/200
1968/1968 [==========================

1968/1968 [==============================] - 22s 11ms/step - loss: 0.0204 - accuracy: 0.9924
Epoch 52/200
1968/1968 [==============================] - 22s 11ms/step - loss: 0.0121 - accuracy: 0.9964
Epoch 53/200
1968/1968 [==============================] - 22s 11ms/step - loss: 0.0253 - accuracy: 0.9898
Epoch 54/200
1968/1968 [==============================] - 22s 11ms/step - loss: 0.0281 - accuracy: 0.9909
Epoch 55/200
1968/1968 [==============================] - 22s 11ms/step - loss: 0.0218 - accuracy: 0.9909
Epoch 56/200
1968/1968 [==============================] - 21s 11ms/step - loss: 0.0198 - accuracy: 0.9914
Epoch 57/200
1968/1968 [==============================] - 22s 11ms/step - loss: 0.0307 - accuracy: 0.9888
Epoch 58/200
1968/1968 [==============================] - 22s 11ms/step - loss: 0.0370 - accuracy: 0.9863
Epoch 59/200
1968/1968 [==============================] - 22s 11ms/step - loss: 0.0149 - accuracy: 0.99493s -
Epoch 60/200
1968/1968 [==============================]

1969/1969 [==============================] - 20s 10ms/step - loss: 0.0147 - accuracy: 0.9954
Epoch 112/200
1969/1969 [==============================] - 21s 11ms/step - loss: 0.0076 - accuracy: 0.9975
Epoch 113/200
1969/1969 [==============================] - 20s 10ms/step - loss: 0.0074 - accuracy: 0.9980
Epoch 114/200
1969/1969 [==============================] - 21s 11ms/step - loss: 0.0211 - accuracy: 0.9919
Epoch 115/200
1969/1969 [==============================] - 20s 10ms/step - loss: 0.0271 - accuracy: 0.9914
Epoch 116/200
1969/1969 [==============================] - 21s 11ms/step - loss: 0.0061 - accuracy: 0.9980
Restoring model weights from the end of the best epoch
Epoch 00116: early stopping
218/218 [==============================] - 2s 9ms/step
---------------------------epoch :  9 ---------------------------
Epoch 1/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.9757 - accuracy: 0.5698
Epoch 2/200
1969/1969 [==============================] - 22s 11

1969/1969 [==============================] - 21s 11ms/step - loss: 0.0262 - accuracy: 0.9909
Epoch 147/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0184 - accuracy: 0.9954
Epoch 148/200
1969/1969 [==============================] - 21s 11ms/step - loss: 0.0125 - accuracy: 0.9944
Epoch 149/200
1969/1969 [==============================] - 23s 12ms/step - loss: 0.0159 - accuracy: 0.9939
Epoch 150/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0285 - accuracy: 0.9909
Epoch 151/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0129 - accuracy: 0.9959
Epoch 152/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0039 - accuracy: 0.9985
Epoch 153/200
1969/1969 [==============================] - 21s 11ms/step - loss: 0.0128 - accuracy: 0.9939
Epoch 154/200
1969/1969 [==============================] - 21s 11ms/step - loss: 0.0193 - accuracy: 0.9929
Epoch 155/200
1969/1969 [==========================

Epoch 141/200
1969/1969 [==============================] - 21s 11ms/step - loss: 0.0112 - accuracy: 0.9939
Epoch 142/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0248 - accuracy: 0.9919
Epoch 143/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0114 - accuracy: 0.9964
Epoch 144/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0340 - accuracy: 0.9888
Epoch 145/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0171 - accuracy: 0.9944
Epoch 146/200
1969/1969 [==============================] - 21s 11ms/step - loss: 0.0072 - accuracy: 0.9970
Epoch 147/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0029 - accuracy: 0.9990
Epoch 148/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0019 - accuracy: 0.9995
Epoch 149/200
1969/1969 [==============================] - 22s 11ms/step - loss: 0.0235 - accuracy: 0.9924
Epoch 150/200
1969/1969 [============

In [14]:
k_accuracy

[[0.49628089116588575, 0.8949771523475647],
 [0.3637524242066357, 0.913241982460022],
 [0.5322655265260176, 0.913241982460022],
 [0.6938118856383241, 0.8949771523475647],
 [0.2776972888607413, 0.922374427318573],
 [0.4885225205524871, 0.8995434045791626],
 [0.4517198034549413, 0.8721461296081543],
 [0.7345364651548754, 0.8669725060462952],
 [0.2453405632885224, 0.9357798099517822],
 [0.7391612801803361, 0.8807339668273926]]

In [15]:
mat

[array([[ 75,   7],
        [ 16, 121]], dtype=int64), array([[ 68,   9],
        [ 10, 132]], dtype=int64), array([[ 73,  14],
        [  5, 127]], dtype=int64), array([[ 62,  17],
        [  6, 134]], dtype=int64), array([[ 75,  12],
        [  5, 127]], dtype=int64), array([[ 75,  16],
        [  6, 122]], dtype=int64), array([[ 74,  16],
        [ 12, 117]], dtype=int64), array([[ 80,  11],
        [ 18, 109]], dtype=int64), array([[ 89,   5],
        [  9, 115]], dtype=int64), array([[ 68,  18],
        [  8, 124]], dtype=int64)]